In [1]:
#Sorry for this crazy import cell... this will be cleaned up

import psycopg2
import nltk
import unicodedata
import pandas as pd
import pprint
import pickle
import re
import os
from nltk.corpus import wordnet
import time

stops = nltk.corpus.stopwords.words('english')
wnl = nltk.WordNetLemmatizer()
nltk.download('averaged_perceptron_tagger')

/anaconda3/lib/python3.7/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/alice.naghshineh/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

https://www.machinelearningplus.com/nlp/lemmatization-examples-python/

In [2]:
#Input your PostGres credentials to connect

dbname = 'Yelp'
username = 'postgres'
host = 'localhost'
password = 'LimathaFaaltaThalik12?'

conn = psycopg2.connect('dbname={} user={} host={} password={}'.format(dbname, username, host, password))
cur = conn.cursor()

In [3]:
cur = conn.cursor()
cur.execute("""
    SELECT * FROM review LIMIT 100000
""")

cols = ['review_id', 'user_id', 'business_id', 'stars', 'review_date', 'review_text', 'useful', 'funny', 'cool']

review_sample = pd.DataFrame(cur.fetchall(), columns=cols)

In [4]:
review_sample

,review_id,user_id,business_id,stars,review_date,review_text,useful,funny,cool
0,f-CFLWI0V6x14sJBT4phtA,jO44Apni7iJZVVK4HQ60tA,VCneJXQYzZdcghJhcrta4A,4,2009-08-24,"I haven't roller skated for 25 years. Yes, I'...",9,14,8
1,YMQB3voexHAfiF4TKF5zrw,L3RkqelOGJBRXICDoDDBQQ,-WfpxcNbp-kgrfOUG6xVvg,4,2014-08-16,Always good solid burgers totally customizable...,0,0,0
2,RvEwyT3Z5-cHdhQWOkllYg,c7pPoQPgSUZERW8XVfV0gQ,kOkoz7_VLyqyN5UPselqoQ,5,2017-04-11,Hello I went there today and had suck a great ...,2,2,2
3,O0KhoPgwUJEXmtWc-Gp2cQ,DeXKbQYNx52OlOizobOLJw,3ZBIqDLVl3W9wIZufxESLA,5,2018-08-08,I've been coming here since I was a wee little...,2,0,1
4,E1HOrQ42ICFtougG9T3JDg,NtzFoCZQ34uiB-n3xWppgw,cxaMso5tqJ5KMCM-JgUbKg,5,2018-07-11,The best calamari i have ever had. Great happy...,0,0,0
5,1PS4VV1V3EGI1GC2xMdpEQ,HFFiM63x9asevqVOZG5GBQ,Eqd5QGaW7_C-qHNGzs1uUg,4,2018-08-17,While its neighbor has changed several times (...,4,1,2
6,EytZKmTmVLecvRTBVhjHWw,ZHRFT-lplvMuFJPX_qe5fA,hDa63HeG4J43pbeE3DUw8w,5,2018-02-14,Highly recommend them. As a CEO of a 100+ emp...,1,0,0
7,S7TTzzqNQ_PolTM4qvFWjg,olFgM-3AuGRbGNSdniI4pg,IPvzhUfrOukhT0i7oBeHkQ,1,2017-05-29,"If I could give them zero stars, I would. I of...",1,1,0
8,88EBQ7OlyywVrpB8eEJhHg,OLoao5V-Tsr-n7A4TZkt4Q,gxEkxkqQrrK62DJnXxXDLg,5,2011-03-28,Fast service. Great burgers and fries. So far ...,1,0,0
9,xiID4o0Rip3py9wOk70vTA,5CgjjDAic2-FAvCtiHpytA,UeRYLA4XT0pFkJednCx-mg,4,2017-01-06,Love this store. It can get pretty hectic on t...,2,0,1


In [7]:
print(review_sample.loc[10, 'review_text'])

Woo Hoo!! The new location is finally open!! 

That said, this location is on Rock Springs right behind the Arby's on Rock Springs and Lake Mead (I-95 and Lake Mead).

The food is nice and fresh, the Katsu is crispy, and the service (just like their other location) is friendly and on point. Whenever I've visited the other location, they've always been consistent with their food and their service. They have the same food offerings as their other location... Chicken Katsu, Poke, Teriyaki Chicken, Kalua Pig, and an assortment of other Hawaiian BBQ staples. Their food is yummy and reasonably priced for the portions you get.

I'm a big fan of their Kimchi slaw, and it's nice not having to drive so far to get it. 

If you haven't tried it before, it's delicious! I'm definitely coming back now that it's practically in my back yard!


Contractions:
https://gist.github.com/J3RN/ed7b420a6ea1d5bd6d06

In [8]:
common_nonneg_contr = ["could've",
"he'd",
"he'd've",
"he'll",
"he's",
"how'd",
"how'll",
"how's",
"i'd",
"i'd've",
"i'll",
"i'm",
"i've",
"it'd",
"it'd've",
"it'll",
"it's",
"let's",
"ma'am",
"might've",
"must've",
"o'clock",
"'ow's'at",
"she'd",
"she'd've",
"she'll",
"she's",
"should've",
"somebody'd",
"somebody'd've",
"somebody'll",
"somebody's",
"someone'd",
"someone'd've",
"someone'll",
"someone's",
"something'd",
"something'd've",
"something'll",
"something's",
"that'll",
"that's",
"there'd",
"there'd've",
"there're",
"there's",
"they'd",
"they'd've",
"they'll",
"they're",
"they've",
"'twas",
"we'd",
"we'd've",
"we'll",
"we're",
"we've",
"what'll",
"what're",
"what's",
"what've",
"when's",
"where'd",
"where's",
"where've",
"who'd",
"who'd've",
"who'll",
"who're",
"who's",
"who've",
"why'll",
"why're",
"why's",
"would've",
"y'all",
"y'all'll",
"y'all'd've",
"you'd",
"you'd've",
"you'll",
"you're",
"you've"]

In [9]:
neg_stops1 = ['no',
 'nor',
 'not',
 'don',
 "don't",
 'ain',
 'aren',
 "aren't",
 'couldn',
 "couldn't",
 'didn',
 "didn't",
 'doesn',
 "doesn't",
 'hadn',
 "hadn't",
 'hasn',
 "hasn't",
 'haven',
 "haven't",
 'isn',
 "isn't",
 'mightn',
 "mightn't",
 'mustn',
 "mustn't",
 'needn',
 "needn't",
 'shan',
 "shan't",
 'shouldn',
 "shouldn't",
 'wasn',
 "wasn't",
 'weren',
 "weren't",
 'won',
 "won't",
 'wouldn',
 "wouldn't",
 'but',
 "don'",
 "ain't"]

words_to_be_added = ['us']
stops1 = ['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each',
 'few',
 'more',
 'most',
 'other',
 'some',
 'such',
 'no',
 'nor',
 'not',
 'only',
 'own',
 'same',
 'so',
 'than',
 'too',
 'very',
 's',
 't',
 'can',
 'will',
 'just',
 'don',
 "don't",
 'should',
 "should've",
 'now',
 'd',
 'll',
 'm',
 'o',
 're',
 've',
 'y',
 'ain',
 'aren',
 "aren't",
 'couldn',
 "couldn't",
 'didn',
 "didn't",
 'doesn',
 "doesn't",
 'hadn',
 "hadn't",
 'hasn',
 "hasn't",
 'haven',
 "haven't",
 'isn',
 "isn't",
 'ma',
 'mightn',
 "mightn't",
 'mustn',
 "mustn't",
 'needn',
 "needn't",
 'shan',
 "shan't",
 'shouldn',
 "shouldn't",
 'wasn',
 "wasn't",
 'weren',
 "weren't",
 'won',
 "won't",
 'wouldn',
 "wouldn't"]


In [10]:
for x in neg_stops1:
    if x in stops1:
        stops1.remove(x)
        
stops1

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each',
 'few

In [11]:
new_stops = stops1 + common_nonneg_contr
list(set(new_stops))

['up',
 "should've",
 'about',
 "she'd've",
 "what'll",
 'under',
 "how'll",
 "y'all",
 'how',
 'other',
 'm',
 'yourselves',
 "it'll",
 "might've",
 'them',
 "who've",
 'can',
 'on',
 "i'd",
 'they',
 "who'd've",
 "would've",
 'her',
 'ma',
 'than',
 'a',
 "what's",
 'by',
 "something'll",
 'it',
 'ourselves',
 "who's",
 'of',
 'the',
 'to',
 "he'll",
 "who'd",
 'any',
 'am',
 'having',
 'for',
 'with',
 'just',
 "i'd've",
 'i',
 'did',
 'there',
 'most',
 'here',
 "there're",
 "they'll",
 'from',
 'where',
 "they'd've",
 'both',
 "somebody'd've",
 "he's",
 "someone's",
 "we'd've",
 'll',
 'is',
 'down',
 'such',
 "why's",
 "it's",
 'once',
 "i've",
 "that'll",
 'be',
 'few',
 'in',
 "he'd",
 "how'd",
 'if',
 've',
 "there'd've",
 "i'm",
 'was',
 "they're",
 "someone'd've",
 'between',
 'through',
 'only',
 'our',
 "she'll",
 're',
 "why'll",
 "why're",
 "could've",
 "somebody'd",
 'herself',
 'your',
 'his',
 'because',
 'until',
 'their',
 'been',
 "let's",
 "where've",
 "you'll",
 

In [ ]:
#Turning this into text, as it's the less efficient code
"""

def _remove_stops(tokens):
    stops = nltk.corpus.stopwords.words('english')
    neg_stops = [
    'no', 'not', 'nor', 'don\'', 'don\'t', 'ain', 
    'ain\'t', 'aren\'t', 'aren', 'couldn', 'couldn\'t', 
    'didn', 'didn\'t', 'doesn', 'doesn\'t', 'hadn', 
    'hadn\'t', 'hasn', 'hasn\'t', 'haven', 'haven\'t',
    'isn', 'isn\'t', 'mightn', 'mightn\'t', 'mustn', 
    'mustn\'t', 'needn', 'needn\'t', 'shan', 'shan\'t',
    'shouldn', 'shouldn\'t', 'wasn', 'wasn\'t', 'weren',
    'weren\'t', 'won', 'won\'t', 'wouldn', 'wouldn\'t'
    ]
#still leaves in but and don.. fix this.. 
#doesn't get rid of other obvious stopwords, like i'm, they're....
    for x in neg_stops:
        if x in stops:
            stops.remove(x)
            
    tokens_without_stops = [token for token in tokens if token not in stops]
    return tokens_without_stops

def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].lower()
    tag_dict = {"a": wordnet.ADJ,
                "n": wordnet.NOUN,
                "v": wordnet.VERB,
                "r": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

#I think I adjusted this properly to remove digits
#however for phrases that are hyphenated, like chilled-to-the-bone, the code returns the token 'chilledtothebone.'
#This will need to be fixed
def _clean_review(text):
    text = text.lower()
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf8', 'ignore')
    text = re.sub(r"[^A-Za-z\s']", '', text)  
    tokens = text.split()
    for i, token in enumerate(tokens):
        tokens[i] = wnl.lemmatize(token, pos= get_wordnet_pos(token))
    return tokens

def _process_review(text):
    tokens = _remove_stops(_clean_review(text))
    return tokens
    
    
    
"""

In [12]:
stops = nltk.corpus.stopwords.words('english')
neg_stops = [
    'no', 'not', 'nor', 'don\'', 'don\'t', 'ain', 
    'ain\'t', 'aren\'t', 'aren', 'couldn', 'couldn\'t', 
    'didn', 'didn\'t', 'doesn', 'doesn\'t', 'hadn', 
    'hadn\'t', 'hasn', 'hasn\'t', 'haven', 'haven\'t',
    'isn', 'isn\'t', 'mightn', 'mightn\'t', 'mustn', 
    'mustn\'t', 'needn', 'needn\'t', 'shan', 'shan\'t',
    'shouldn', 'shouldn\'t', 'wasn', 'wasn\'t', 'weren',
    'weren\'t', 'won', 'won\'t', 'wouldn', 'wouldn\'t'
    ]
#still leaves in but and don.. fix this.. 
#doesn't get rid of other obvious stopwords, like i'm, they're....
for x in neg_stops:
    if x in stops:
        stops.remove(x)
        
def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].lower()
    tag_dict = {"a": wordnet.ADJ,
                "n": wordnet.NOUN,
                "v": wordnet.VERB,
                "r": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)


def _clean_review2(text):
    text = text.lower()
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf8', 'ignore')
    text = re.sub(r"[^A-Za-z\s']", '', text)  
    tokens = [token for token in text.split() if token not in new_stops]
    for i, token in enumerate(tokens):
        tokens[i] = wnl.lemmatize(token, pos= get_wordnet_pos(token))
    return tokens

def _process_review2(text):
    tokens = _remove_stops(_clean_review2(text))
    return tokens

In [ ]:
#Turning this into text, as it is less efficient than 2

"""

stops = nltk.corpus.stopwords.words('english')
neg_stops = [
    'no', 'not', 'nor', 'don\'', 'don\'t', 'ain', 
    'ain\'t', 'aren\'t', 'aren', 'couldn', 'couldn\'t', 
    'didn', 'didn\'t', 'doesn', 'doesn\'t', 'hadn', 
    'hadn\'t', 'hasn', 'hasn\'t', 'haven', 'haven\'t',
    'isn', 'isn\'t', 'mightn', 'mightn\'t', 'mustn', 
    'mustn\'t', 'needn', 'needn\'t', 'shan', 'shan\'t',
    'shouldn', 'shouldn\'t', 'wasn', 'wasn\'t', 'weren',
    'weren\'t', 'won', 'won\'t', 'wouldn', 'wouldn\'t'
    ]
#still leaves in but and don.. fix this.. 
#doesn't get rid of other obvious stopwords, like i'm, they're....
for x in neg_stops:
    if x in stops:
        stops.remove(x)
        
def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].lower()
    tag_dict = {"a": wordnet.ADJ,
                "n": wordnet.NOUN,
                "v": wordnet.VERB,
                "r": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)


def _clean_review3(text):
    text = text.lower()
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf8', 'ignore')
    text = re.sub(r"[^A-Za-z\s']", '', text)
    tokens = [wnl.lemmatize(token, pos = get_wordnet_pos(token)) for token in text.split() if token not in stops]
    return tokens

def _process_review3(text):
    tokens = _remove_stops(_clean_review2(text))
    return tokens
    
    
"""

In [ ]:
#This code is slow

"""

stops = nltk.corpus.stopwords.words('english')
neg_stops = [
    'no', 'not', 'nor', 'don\'', 'don\'t', 'ain', 
    'ain\'t', 'aren\'t', 'aren', 'couldn', 'couldn\'t', 
    'didn', 'didn\'t', 'doesn', 'doesn\'t', 'hadn', 
    'hadn\'t', 'hasn', 'hasn\'t', 'haven', 'haven\'t',
    'isn', 'isn\'t', 'mightn', 'mightn\'t', 'mustn', 
    'mustn\'t', 'needn', 'needn\'t', 'shan', 'shan\'t',
    'shouldn', 'shouldn\'t', 'wasn', 'wasn\'t', 'weren',
    'weren\'t', 'won', 'won\'t', 'wouldn', 'wouldn\'t'
    ]
#still leaves in but and don.. fix this.. 
#doesn't get rid of other obvious stopwords, like i'm, they're....
for x in neg_stops:
    if x in stops:
        stops.remove(x)
        
def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].lower()
    tag_dict = {"a": wordnet.ADJ,
                "n": wordnet.NOUN,
                "v": wordnet.VERB,
                "r": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)


def _clean_review4(text):
    text = text.lower()
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf8', 'ignore')
    text = re.sub(r"[^A-Za-z\s']", '', text)  
    tokens = [wnl.lemmatize(token, pos= get_wordnet_pos(token)) for token in text.split()]
    tokens = [token for token in tokens if token not in stops]
    return tokens

def _process_review4(text):
    tokens = _remove_stops(_clean_review2(text))
    return tokens
    
"""

In [ ]:
"""

stops = nltk.corpus.stopwords.words('english')
neg_stops = [
    'no', 'not', 'nor', 'don\'', 'don\'t', 'ain', 
    'ain\'t', 'aren\'t', 'aren', 'couldn', 'couldn\'t', 
    'didn', 'didn\'t', 'doesn', 'doesn\'t', 'hadn', 
    'hadn\'t', 'hasn', 'hasn\'t', 'haven', 'haven\'t',
    'isn', 'isn\'t', 'mightn', 'mightn\'t', 'mustn', 
    'mustn\'t', 'needn', 'needn\'t', 'shan', 'shan\'t',
    'shouldn', 'shouldn\'t', 'wasn', 'wasn\'t', 'weren',
    'weren\'t', 'won', 'won\'t', 'wouldn', 'wouldn\'t'
    ]
#still leaves in but and don.. fix this.. 
#doesn't get rid of other obvious stopwords, like i'm, they're....
for x in neg_stops:
    if x in stops:
        stops.remove(x)
        
def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].lower()
    tag_dict = {"a": wordnet.ADJ,
                "n": wordnet.NOUN,
                "v": wordnet.VERB,
                "r": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)


def _clean_review5(text):
    text = text.lower()
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf8', 'ignore')
    text = re.sub(r"[^A-Za-z\s']", '', text)
    tokens = text.split()
    for i, token in enumerate(tokens):
        tokens[i] = wnl.lemmatize(token, pos= get_wordnet_pos(token))
    tokens = [token for token in tokens if token not in stops]
    return tokens

def _process_review5(text):
    tokens = _remove_stops(_clean_review2(text))
    return tokens
    
"""

In [13]:
_clean_review2(review_sample.loc[10, 'review_text'])

['woo',
 'hoo',
 'new',
 'location',
 'finally',
 'open',
 'say',
 'location',
 'rock',
 'spring',
 'right',
 'behind',
 "arby's",
 'rock',
 'spring',
 'lake',
 'mead',
 'lake',
 'mead',
 'food',
 'nice',
 'fresh',
 'katsu',
 'crispy',
 'service',
 'like',
 'location',
 'friendly',
 'point',
 'whenever',
 'visit',
 'location',
 'always',
 'consistent',
 'food',
 'service',
 'food',
 'offering',
 'location',
 'chicken',
 'katsu',
 'poke',
 'teriyaki',
 'chicken',
 'kalua',
 'pig',
 'assortment',
 'hawaiian',
 'bbq',
 'staple',
 'food',
 'yummy',
 'reasonably',
 'price',
 'portion',
 'get',
 'big',
 'fan',
 'kimchi',
 'slaw',
 'nice',
 'not',
 'drive',
 'far',
 'get',
 "haven't",
 'try',
 'delicious',
 'definitely',
 'come',
 'back',
 'practically',
 'back',
 'yard']

In [14]:
def apply_on_column(data):
    data['review_text'] = data['review_text'].apply(lambda x: _clean_review2(x))
    return data

In [ ]:
def map_on_column(data):
    data['review_text'] = data['review_text'].map(lambda x: _clean_review2(x))
    return data

In [15]:
#Takes about 8 min 8 sec for 50000 (_clean_review2)
# 15 min 24.965 seconds for 100000
start = time.time()
apply_on_column(review_sample)
end = time.time()
dur = end - start
# Verify that the function is working
print('Processed {} instances in {} minutes {} seconds.\n'.format(review_sample.shape[0], dur//60, dur%60))

Processed 100000 instances in 15.0 minutes 35.78048801422119 seconds.



In [16]:
review_sample.loc[27, 'review_text']

['chocolate',
 'ice',
 'cream',
 'caramel',
 'oh',
 'delight',
 'fill',
 'hand',
 'make',
 'creation',
 'delightfully',
 'delicious',
 'proportion',
 'ice',
 'cream',
 'make',
 'house',
 'far',
 'try',
 'rhubarb',
 'surprisingly',
 'tart',
 'ice',
 'cream',
 'wonderful',
 'rhubarb',
 'flavour',
 'second',
 'visit',
 'strawberry',
 'hand',
 'best',
 'strawberry',
 'ice',
 'cream',
 'ever',
 'make',
 'local',
 'strawberry',
 'cooked',
 'make',
 'ice',
 'cream',
 'day',
 'freshness',
 'incredible',
 'flavour',
 'delicious',
 'savour',
 'every',
 'bit',
 'actually',
 'chocolate',
 'inspire',
 'visit',
 'delight',
 'sell',
 'raspberry',
 'jam',
 'fill',
 'dark',
 'chocolate',
 "hubby's",
 'favourite',
 'but',
 'hard',
 'find',
 'bought',
 'box',
 'selection',
 'offer',
 'day',
 'include',
 'ginger',
 'adore',
 'caramel',
 'smoke',
 'sea',
 'salt',
 'also',
 'hit',
 'chili',
 'lemon',
 'surprisingly',
 'spicy',
 'but',
 'tasty',
 'peanut',
 'butter',
 'must',
 'try',
 'peanut',
 'butter',
 '

In [ ]:
def iterrows_at(data):
    for i, row in data.iterrows():
        data.at[i, 'review_text'] = _clean_review2(data.at[i, 'review_text'])

In [ ]:
#Takes a little longer.. like 7 min 56 seconds for 50000 reviews (_clean_review2)
start = time.time()
iterrows_at(review_sample)
end = time.time()
dur = end - start
# Verify that the function is working
print('Processed {} instances in {} minutes {} seconds.\n'.format(review_sample.shape[0], dur//60, dur%60))

In [ ]:
#7 min 31.66 seconds
#15 min 50.93 seconds for 100000
start = time.time()
map_on_column(review_sample)
end = time.time()
dur = end - start
# Verify that the function is working
print('Processed {} instances in {} minutes {} seconds.\n'.format(review_sample.shape[0], dur//60, dur%60))

In [ ]:
review_sample